# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Simplex

In [1]:
import time

In [2]:
import numpy as np

# Algorithm

In [3]:
%%time
def simplex(c, A, b):
    table = initialize(c, A, b)
    while not search_optimum(table):
        pass
    return solution(c, table)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.4 µs


In [4]:
%%time
def initialize(c, A, b):
    (m, n), k = A.shape, len(c)

    # simplex table:
    # |A|E|b|
    # |c|0|0|
    table = np.zeros((m + 1, m + n + 1))
    table[:m, :n] = A
    table[range(m), range(n, n + m)] = 1
    table[:-1, -1] = b
    table[-1, :k] = c

    return table

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 13.4 µs


In [5]:
%%time
def search_optimum(table):
    index = np.argwhere(table[-1, :-1] > 0).ravel()

    # optimum found
    if not len(index):
        return True

    # pivotal column
    j = index[0]
    column = table[:-1, j].copy()
    column[column <= 0] = -1

    if np.all(column <= 0):
        raise ArithmeticError('the system is unbounded')

    # pivotal row
    pivots = table[:-1, -1] / column
    pivots[column <= 0] = np.inf
    i = np.argmin(pivots).ravel()[0]

    # eliminate by pivot at (i, j)
    row = table[i] / table[i][j]
    table[:] -= np.outer(table[:, j], row)
    table[i, :] = row
    table[:, j] = table[:, j].round()

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.06 µs


In [6]:
%%time
def solution(c, table):
    (m, n), k = table.shape, len(c)

    # pivotal columns
    s = np.sum(table == 0, axis=0) == m - 1
    t = np.sum(table == 1, axis=0) == 1

    # solution
    x = np.zeros(n - 1)

    for j in range(n - 1):
        if s[j] and t[j]:
            x[j] = table[:, j] @ table[:, -1]

    return dict(
        x=x[:k],
        slack=x[k:],
        max=-table[-1, -1],
        table=table,
    )

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.63 µs


# Run

## Linear Program #1

maximize: -x + 3y + 2z

subject to:
x + y + z <= 6
x     + z <= 4
    y + z <= 3
x + y     <= 2

x, y, z >= 0

In [7]:
%%time
c = np.array([-1, 3, 2])
A = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 0],
])
b = np.array([6, 4, 3, 2])

CPU times: user 0 ns, sys: 64 µs, total: 64 µs
Wall time: 67.7 µs


In [8]:
%%time
lp = simplex(c, A, b)

for k in ['x', 'slack', 'table', 'max']:
    print(k, '\n', lp[k], '\n')

x 
 [0. 2. 1.] 

slack 
 [3. 3. 0. 0.] 

table 
 [[ 1.  0.  0.  1.  0. -1.  0.  3.]
 [ 2.  0.  0.  0.  1. -1.  1.  3.]
 [-1.  0.  1.  0.  0.  1. -1.  1.]
 [ 1.  1.  0.  0.  0.  0.  1.  2.]
 [-2.  0.  0.  0.  0. -2. -1. -8.]] 

max 
 8.0 

CPU times: user 2.68 ms, sys: 960 µs, total: 3.64 ms
Wall time: 6.53 ms


## Linear Program #2

maximize: 2r + 4s + 3t + u

subject to:
3r +  s +  t + 4u <= 12
 r - 3s + 2t + 3u <= 7
2r +  s + 3t -  u <= 10

r, s, t, u >= 0

In [9]:
%%time
c = np.array([2, 4, 3, 1])
A = np.array([
    [3, 1, 1, 4],
    [1, -3, 2, 3],
    [2, 1, 3, -1]
])
b = np.array([12, 7, 10])

CPU times: user 42 µs, sys: 5 µs, total: 47 µs
Wall time: 50.3 µs


In [10]:
%%time
lp = simplex(c, A, b)

for k in ['x', 'slack', 'table', 'max']:
    print(k, '\n', lp[k], '\n')

x 
 [ 0.  10.4  0.   0.4] 

slack 
 [ 0. 37.  0.] 

table 
 [[  0.2   0.   -0.4   1.    0.2   0.   -0.2   0.4]
 [  7.    0.   11.    0.    0.    1.    3.   37. ]
 [  2.2   1.    2.6   0.    0.2   0.    0.8  10.4]
 [ -7.    0.   -7.    0.   -1.    0.   -3.  -42. ]] 

max 
 42.0 

CPU times: user 1.91 ms, sys: 0 ns, total: 1.91 ms
Wall time: 1.92 ms


# The End